In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import data as DATADATA
import keras.saving
from keras.models import Sequential, load_model
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam


def build_uncertainty_model():
    inputs = Input(shape=(60, 1))
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(64, return_sequences=False)(x)
    x = Dense(25, activation='relu')(x)
    outputs = Dense(2)(x)  # [mean, log_variance]
    return Model(inputs, outputs)

from keras.models import clone_model
from sklearn.preprocessing import MinMaxScaler
import os
import tensorflow as tf

@keras.saving.register_keras_serializable()
def gaussian_nll(y_true, y_pred):
    mean = y_pred[:, 0]
    log_var = y_pred[:, 1]
    precision = tf.exp(-log_var)
    return tf.reduce_mean(0.5 * (log_var + tf.square(y_true - mean) * precision))


os.makedirs('models-KaggleSMA', exist_ok=True)

tickers = ['SOLUSDT', 'BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT']
scalers = {}  # Save scalers per ticker for later testing

# Step 1: Build combined training dataset
combined_x_train = []
combined_y_train = []

for ticker in tickers:
    df = DATADATA.load_asset(ticker, sampling='1d')
    df = DATADATA.subset(df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2024-03-01'))
    df['Returns'] = df['Close'].pct_change()
    df.dropna(inplace=True)

    data = df.filter(['Returns'])
    dataset = data.values

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    scalers[ticker] = scaler

    for i in range(60, len(scaled_data)):
        combined_x_train.append(scaled_data[i - 60:i, 0])
        combined_y_train.append(scaled_data[i, 0])

# Convert combined dataset to numpy arrays
combined_x_train = np.array(combined_x_train)
combined_y_train = np.array(combined_y_train)
combined_x_train = np.reshape(combined_x_train, (combined_x_train.shape[0], combined_x_train.shape[1], 1))

# Step 2: Train base model
base_model = build_uncertainty_model()
base_model.compile(optimizer='adam', loss=gaussian_nll)
base_model.fit(combined_x_train, combined_y_train, batch_size=8, epochs=16)
base_model.save('models-KaggleSMA/base_model_uncertainty.keras')


Epoch 1/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: -1.2297
Epoch 2/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.8718
Epoch 3/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.8796
Epoch 4/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.8886
Epoch 5/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.9372
Epoch 6/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.9377
Epoch 7/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: -1.9417
Epoch 8/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.9796
Epoch 9/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: -1.9529
Epoch 10/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: -1.9335
Epoch 11/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.8458
Epoch 12/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: -1.9740
Epoch 13/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: -1.9557
Epoch 14/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: -1.9241
Epoch 15/16
457/457 ━━━━━━━━━

In [5]:
from tensorflow.keras.optimizers import Adam


# Step 3: Fine-tune separate model per ticker
for ticker in tickers:
    df = DATADATA.load_asset(ticker, sampling='1d')
    df = DATADATA.subset(df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2024-03-01'))
    df['Returns'] = df['Close'].pct_change()
    df.dropna(inplace=True)

    data = df.filter(['Returns'])
    dataset = data.values

    scaler = scalers[ticker]
    scaled_data = scaler.transform(dataset)

    x_train = []
    y_train = []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i - 60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Clone base model architecture and load weights
    fine_tuned_model = clone_model(base_model)
    fine_tuned_model.set_weights(base_model.get_weights())

    # Freeze base layers
    for layer in fine_tuned_model.layers:
        layer.trainable = False

    # Add a new trainable output layer
    model = build_uncertainty_model()
    model.set_weights(base_model.get_weights())
    model.compile(optimizer='adam', loss=gaussian_nll)
    model.fit(x_train, y_train, batch_size=8, epochs=8)

    model.save(f'models-KaggleSMA/lstm_uncertainty_model_{ticker}.keras')

    # Predict mean and stddev
    preds = model.predict(x_train)
    pred_mean = preds[:, 0]
    pred_std = np.sqrt(np.exp(preds[:, 1]))

    pred_mean_rescaled = scaler.inverse_transform(pred_mean.reshape(-1, 1))

    valid = data[60:].copy()
    valid['PredictedMean'] = pred_mean_rescaled
    valid['PredictedStd'] = pred_std
    valid['Z-Score'] = (valid['Returns'] - valid['PredictedMean']) / valid['PredictedStd']
    display(valid.head(10))



Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: -2.0128
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.0579
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.0196
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.9768
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.0830
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.0264
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.9569
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.9681
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-03,-0.050258,0.021467,0.074299,-0.965355
2022-03-04,-0.076344,0.021570,0.077394,-1.265134
2022-03-05,0.014832,0.020760,0.081077,-0.073107
2022-03-06,-0.056454,0.019549,0.080390,-0.945435
2022-03-07,-0.034409,0.019975,0.081810,-0.664761
2022-03-08,0.008082,0.018945,0.081534,-0.133227
2022-03-09,0.069728,0.017867,0.079010,0.656382
2022-03-10,-0.058937,0.017670,0.074523,-1.027968
2022-03-11,-0.026548,0.018670,0.075445,-0.599349


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: -1.7573
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.7514
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.8577
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.8217
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.8808
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.8551
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.7575
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.7732
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-03,-0.032784,-0.008515,0.090990,-0.266716
2022-03-04,-0.077857,-0.011838,0.093891,-0.703147
2022-03-05,0.006368,-0.013420,0.099403,0.199067
2022-03-06,-0.024802,-0.017715,0.092797,-0.076368
2022-03-07,-0.011265,-0.013901,0.093781,0.028105
2022-03-08,0.019549,-0.014019,0.092808,0.361689
2022-03-09,0.082908,-0.014042,0.089875,1.078712
2022-03-10,-0.060076,-0.012444,0.084576,-0.563197
2022-03-11,-0.017565,-0.007666,0.094306,-0.104966


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: -1.6043
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.8039
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.8063
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.7656
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.8157
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.8412
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.8064
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -1.7661
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-03,-0.038445,0.016750,0.086769,-0.636116
2022-03-04,-0.074767,0.013951,0.093348,-0.950403
2022-03-05,0.016418,0.011821,0.102592,0.044814
2022-03-06,-0.042722,0.006218,0.092309,-0.530178
2022-03-07,-0.023584,0.011467,0.094954,-0.369137
2022-03-08,0.034061,0.011558,0.093966,0.239483
2022-03-09,0.058623,0.010168,0.085407,0.567340
2022-03-10,-0.044107,0.013059,0.079522,-0.718878
2022-03-11,-0.019120,0.018429,0.089755,-0.418353


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: -2.3022
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.4130
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.4651
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.5569
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.4195
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.6457
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -1.0268
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.2674
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-03,-0.022375,-0.000533,0.060350,-0.361936
2022-03-04,-0.051630,-0.001729,0.060231,-0.828485
2022-03-05,0.057949,-0.001774,0.060267,0.990962
2022-03-06,-0.038329,-0.005893,0.059431,-0.545764
2022-03-07,-0.007447,-0.000404,0.060399,-0.116613
2022-03-08,0.001389,-0.002862,0.059970,0.070899
2022-03-09,0.062578,-0.002898,0.059989,1.091467
2022-03-10,-0.038783,-0.004034,0.059666,-0.582401
2022-03-11,0.090749,0.000729,0.060570,1.486209


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: -1.9789
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.0087
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.0946
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: -2.0403
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.1105
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.1876
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.1020
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: -2.1356
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


,Returns,PredictedMean,PredictedStd,Z-Score
Open time,,,,
2022-03-03,-0.024812,0.005881,0.071402,-0.429853
2022-03-04,-0.054742,0.003176,0.071566,-0.809301
2022-03-05,0.019576,-0.000462,0.070856,0.282802
2022-03-06,-0.034400,-0.011925,0.067029,-0.335308
2022-03-07,-0.033140,-0.004069,0.068095,-0.426925
2022-03-08,0.002571,-0.002110,0.069043,0.067792
2022-03-09,0.038462,-0.003548,0.068991,0.608917
2022-03-10,-0.037860,-0.002063,0.069332,-0.516315
2022-03-11,-0.011976,0.009957,0.073332,-0.299095
